In [1]:
import os
import cv2
import imghdr
import tensorflow as tf

In [4]:
# Code to clean images

data_dir = 'data' 
image_exts = ['jpeg','jpg', 'bmp', 'png']
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

In [14]:
# Load the dataset using image_dataset_from_directory
data = tf.keras.utils.image_dataset_from_directory('data')
data = data.map(lambda x,y: (x/255, y))
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

Found 1116 files belonging to 10 classes.


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [3]:
model = Sequential()

model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 16)       0

In [32]:
# Define the number of epochs
epochs = 20

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs
)

Epoch 1/20
28/28 [==============================] - 14s 449ms/step - loss: 0.5250 - accuracy: 0.8337 - val_loss: 0.4017 - val_accuracy: 0.8909
Epoch 2/20
28/28 [==============================] - 13s 435ms/step - loss: 0.5058 - accuracy: 0.8382 - val_loss: 0.4064 - val_accuracy: 0.9045
Epoch 3/20
28/28 [==============================] - 14s 488ms/step - loss: 0.4465 - accuracy: 0.8605 - val_loss: 0.2959 - val_accuracy: 0.9545
Epoch 4/20
28/28 [==============================] - 15s 504ms/step - loss: 0.4052 - accuracy: 0.8772 - val_loss: 0.3098 - val_accuracy: 0.9273
Epoch 5/20
28/28 [==============================] - 15s 494ms/step - loss: 0.3698 - accuracy: 0.8817 - val_loss: 0.2895 - val_accuracy: 0.9500
Epoch 6/20
28/28 [==============================] - 14s 484ms/step - loss: 0.3593 - accuracy: 0.8839 - val_loss: 0.2551 - val_accuracy: 0.9636
Epoch 7/20
28/28 [==============================] - 14s 471ms/step - loss: 0.3107 - accuracy: 0.9196 - val_loss: 0.2422 - val_accuracy: 0.9545

In [33]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_dataset)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

7/7 [==============================] - 6s 103ms/step - loss: 0.1011 - accuracy: 0.9864
Validation Loss: 0.1010923907160759
Validation Accuracy: 0.9863636493682861


In [34]:
# Save the model
model.save("product_classification_cnn.h5")

In [35]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the saved model
model = tf.keras.models.load_model("product_classification_cnn.h5")

# Load and preprocess the new image
img_path = 'data/SPOTTY BUNTING/71FLL7rTUJL.jpg'
img = image.load_img(img_path, target_size=(img_height, img_width))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Normalize pixel values

# Make a prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions, axis=1)
print(predicted_class)
# Map the predicted class index to the label
class_labels = dataset.class_names  # Get class names from the dataset
print(class_labels)
predicted_label = class_labels[predicted_class[0]]

print(f"Predicted Label: {predicted_label}")

1/1 [==============================] - 0s 333ms/step
[9]
['6 RIBBONS RUSTIC CHARM', 'ALARM CLOCK BAKELIKE RED', 'CHOCOLATE HOT WATER BOTTLE', 'JUMBO STORAGE BAG SUKI', 'LUNCH BAG PINK POLKADOT', 'LUNCH BAG WOODLAND', 'REGENCY CAKESTAND 3 TIER', 'RETROSPOT TEA SET CERAMIC 11 PC', 'REX CASHCARRY JUMBO SHOPPER', 'SPOTTY BUNTING']
Predicted Label: SPOTTY BUNTING
